In [10]:
from openpharmacophore.screening.alignment import *
from openpharmacophore.pharmacophore import Pharmacophore
from openpharmacophore import pharmacophoric_elements
import pyunitwizard as puw

from rdkit import Chem, RDLogger, RDConfig
from rdkit.Chem import ChemicalFeatures, rdDistGeom
from rdkit.Chem.Pharm3D import EmbedLib
RDLogger.DisableLog('rdApp.*')

import bisect
from operator import itemgetter
import os

In [16]:
def align_molecules(pharmacophore, molecules, verbose=0):
    """ Align a list of molecules to a given pharmacophore.

    Parameters
    ----------
    molecules: list of rdkit.Chem.mol
        List of molecules to align.

    verbose: int
        Level of verbosity


    """
    n_molecules = len(molecules)
    n_matches = 0
    n_fails = 0
    aligned_mols = []
    
    rdkit_pharmacophore, radii = pharmacophore.to_rdkit()
    apply_radii_to_bounds(radii, rdkit_pharmacophore)

    fdef = os.path.join(RDConfig.RDDataDir,'BaseFeatures.fdef')
    featFactory = ChemicalFeatures.BuildFeatureFactory(fdef)

    for i, mol in enumerate(molecules):

        if verbose == 1 and i % 100 == 0 and i != 0:
            print(f"Screened {i} molecules. Number of matches: {n_matches}; Number of fails: {n_fails}")

        bounds_matrix = rdDistGeom.GetMoleculeBoundsMatrix(mol)
        # Check if the molecule features can match with the pharmacophore.
        can_match, all_matches = EmbedLib.MatchPharmacophoreToMol(mol, featFactory, rdkit_pharmacophore)
        # all_matches is a list of tuples where each tuple contains the chemical features
        if can_match:
            # Match the molecule to the pharmacophore without aligning it
            failed, bounds_matrix_matched, matched_mols, match_details = EmbedLib.MatchPharmacophore(all_matches, 
                                                                                            bounds_matrix,
                                                                                            rdkit_pharmacophore, 
                                                                                            useDownsampling=True)
            if failed:
                if verbose == 2:
                    print(f"Couldn't embed molecule {i}")
                n_fails += 1
                continue
        else:
            if verbose == 2:
                print(f"Couldn't match molecule {i}")
            n_fails += 1
            continue
        atom_match = [list(x.GetAtomIds()) for x in matched_mols]
        try:
            mol_H = Chem.AddHs(mol)
            # Embed molecule onto the pharmacophore
            # embeddings is a list of molecules with a single conformer
            b_matrix, embeddings, num_fail = EmbedLib.EmbedPharmacophore(mol_H, atom_match, rdkit_pharmacophore, count=10)
        except Exception as e:
            if verbose == 2:
                print(e)
                print (f"Bounds smoothing failed for molecule {i}")
            n_fails += 1
            continue
        # Align embeddings to the pharmacophore 
        SSDs = transform_embeddings(rdkit_pharmacophore, embeddings, atom_match) 
        best_fit_index = min(enumerate(SSDs), key=itemgetter(1))[0]
        
        try:
            mol_id = mol.GetProp("_Name")
        except:
            mol_id = None
        matched_mol = (SSDs[best_fit_index], mol_id, embeddings[best_fit_index])
        # Append to list in ordered manner
        try:
            bisect.insort(aligned_mols, matched_mol) 
            n_matches += 1
        except:
            # Case when a molecule is repeated. It will throw an error since bisect
            # cannot compare molecules.
            n_molecules -= 1
            continue
        
    return aligned_mols, n_matches, n_fails, n_molecules

In [3]:
elements = [
    pharmacophoric_elements.HBAcceptorSphere(center=puw.quantity([3.877, 7.014, 1.448], "angstroms"),
                                             radius=puw.quantity(1.0, "angstroms")),
    pharmacophoric_elements.HBAcceptorSphere(center=puw.quantity([7.22, 11.077, 5.625], "angstroms"),
                                             radius=puw.quantity(1.0, "angstroms")),
    pharmacophoric_elements.HBDonorSphere(center=puw.quantity([4.778, 8.432, 7.805], "angstroms"),
                                         radius=puw.quantity(1.0, "angstroms")),
    pharmacophoric_elements.AromaticRingSphere(center=puw.quantity([1.56433333333334, 7.06399999999999, 3.135], "angstroms"),
                                              radius=puw.quantity(1.0, "angstroms"))
]

pharmacophore = Pharmacophore(elements)
pharmacophore

Pharmacophore(n_elements: 4)

In [4]:
mols = ['Cc1cccc(c2n[nH]cc2c3ccc4ncccc4n3)n1',
      'Cc1cccnc1c2nc(N)sc2c3nc4cccnc4cc3',
      'Cc1cccc(c2[nH]c(CNc5cc(C(=O)N)ccc5)nc2c3ccc4nccnc4c3)n1',
      'Clc1cccc(c2nc(N)sc2c3ccc4ncccc4n3)c1',
      'n1ccccc1c2nn3CCCc3c2c4ccnc5cc(NC(=O)NCCN(C)C)ccc45']

mols = [Chem.MolFromSmiles(mol) for mol in mols]

In [17]:
aligned_mols, n_matches, n_fails, n_molecules = align_molecules(pharmacophore, mols, verbose=2)

Couldn't embed molecule 3


In [18]:
print(f"{n_molecules} molecules in total")
print(f"{n_matches} matches")
print(f"{n_fails} fails")

5 molecules in total
4 matches
1 fails


In [19]:
aligned_mols

[(4.70411856635215, None, <rdkit.Chem.rdchem.Mol at 0x7f1c9922ec70>),
 (5.368822003302398, None, <rdkit.Chem.rdchem.Mol at 0x7f1c991a31f0>),
 (5.890963357713666, None, <rdkit.Chem.rdchem.Mol at 0x7f1c991a3c30>),
 (6.35723689660432, None, <rdkit.Chem.rdchem.Mol at 0x7f1c991a3770>)]

In [33]:
def align_single_molecule(pharmacophore, mol, verbose=0):
    """ Align a list of molecules to a given pharmacophore.

    Parameters
    ----------
    molecules: list of rdkit.Chem.mol
        List of molecules to align.

    verbose: int
        Level of verbosity


    """
    rdkit_pharmacophore, radii = pharmacophore.to_rdkit()
    apply_radii_to_bounds(radii, rdkit_pharmacophore)

    fdef = os.path.join(RDConfig.RDDataDir,'BaseFeatures.fdef')
    featFactory = ChemicalFeatures.BuildFeatureFactory(fdef)

    bounds_matrix = rdDistGeom.GetMoleculeBoundsMatrix(mol)
    # Check if the molecule features can match with the pharmacophore.
    can_match, all_matches = EmbedLib.MatchPharmacophoreToMol(mol, featFactory, rdkit_pharmacophore)
    # all_matches is a list of tuples where each tuple contains the chemical features
    print("All matches ", all_matches)
    failed, bounds_matrix_matched, matched_mols, match_details = EmbedLib.MatchPharmacophore(all_matches, 
                                                                                    bounds_matrix,
                                                                                    rdkit_pharmacophore, 
                                                                                    useDownsampling=True)
    print("\n\nMatched mols ", matched_mols)
    atom_match = [list(x.GetAtomIds()) for x in matched_mols]
    mol_H = Chem.AddHs(mol)
    b_matrix, embeddings, num_fail = EmbedLib.EmbedPharmacophore(mol_H, atom_match, rdkit_pharmacophore, count=10)
    # Align embeddings to the pharmacophore 
    SSDs = transform_embeddings(rdkit_pharmacophore, embeddings, atom_match) 
    best_fit_index = min(enumerate(SSDs), key=itemgetter(1))[0]

    matched_mol = (SSDs[best_fit_index], embeddings[best_fit_index])
    
    return matched_mol

In [34]:
align_single_molecule(pharmacophore, mols[2], verbose=0)

All matches  [(<rdkit.Chem.rdMolChemicalFeatures.MolChemicalFeature object at 0x7f1c98eea1b0>, <rdkit.Chem.rdMolChemicalFeatures.MolChemicalFeature object at 0x7f1c98eea2d0>, <rdkit.Chem.rdMolChemicalFeatures.MolChemicalFeature object at 0x7f1c98eea3f0>, <rdkit.Chem.rdMolChemicalFeatures.MolChemicalFeature object at 0x7f1c98eea510>, <rdkit.Chem.rdMolChemicalFeatures.MolChemicalFeature object at 0x7f1c98eea630>), (<rdkit.Chem.rdMolChemicalFeatures.MolChemicalFeature object at 0x7f1c98eea1b0>, <rdkit.Chem.rdMolChemicalFeatures.MolChemicalFeature object at 0x7f1c98eea2d0>, <rdkit.Chem.rdMolChemicalFeatures.MolChemicalFeature object at 0x7f1c98eea3f0>, <rdkit.Chem.rdMolChemicalFeatures.MolChemicalFeature object at 0x7f1c98eea510>, <rdkit.Chem.rdMolChemicalFeatures.MolChemicalFeature object at 0x7f1c98eea630>), (<rdkit.Chem.rdMolChemicalFeatures.MolChemicalFeature object at 0x7f1c98eea750>, <rdkit.Chem.rdMolChemicalFeatures.MolChemicalFeature object at 0x7f1c98eea870>, <rdkit.Chem.rdMolChem

(5.368822003302398, <rdkit.Chem.rdchem.Mol at 0x7f1c98ee2570>)